In [33]:
from pymongo import MongoClient
import json

In [34]:
c=MongoClient()
oa=c["openalexco"]["works"]
kh=c["kahi_dev"]["works"]
khp=c["kahi_dev"]["person"]

In [54]:
pipeline = [
    {"$match":{"$and":[{"updated.source":"openalex"},{"updated.source":"ciarp"}]}},
    {"$unwind": "$authors"},

    {
        "$addFields": {
            "authors.affiliations": {
                "$filter": {
                    "input": "$authors.affiliations",
                    "as": "aff",
                    "cond": {
                        "$in": [
                            {"source": "ror", "type": "Education"},
                            "$$aff.types"
                        ]
                    }
                }
            }
        }
    },

    {
        "$match": {
            "$expr": {"$gt": [{"$size": "$authors.affiliations"}, 1]}
        }
    },

    {"$replaceRoot": {"newRoot": "$$ROOT"}},
    {
        "$project": {
            "_id": 1,
            "authors": 1,
            "titles": 1,
            "external_ids": {
                "$filter": {
                    "input": "$external_ids",
                    "as": "ext",
                    "cond": {"$eq": ["$$ext.source", "openalex"]}
                }
            }
        }
    }

]

ws=kh.aggregate(pipeline)

In [55]:
ws=list(ws)

In [56]:
len(ws)

103025

In [57]:
ws[0]

{'_id': ObjectId('68ade992076e24eb542696b0'),
 'titles': [{'title': 'Incorporating Context-Awareness Services in Adaptive U-MAS Learning Environments',
   'lang': 'en',
   'source': 'openalex'},
  {'title': 'Incorporating context-awareness services in adaptive U-MAS learning environments',
   'lang': 'en',
   'source': 'ciarp',
   'provenance': 'ciarp'},
  {'title': 'Incorporating context-awareness services in adaptive U-MAS learning environments',
   'lang': 'en',
   'source': 'scholar'},
  {'title': 'Incorporating Context-Awareness Services in Adaptive U-MAS Learning Environments',
   'lang': 'en',
   'source': 'minciencias'}],
 'authors': {'id': 'A5061987291',
  'full_name': 'Néstor Darío Duque',
  'affiliations': [{'id': '031n6w191',
    'name': 'Universidad de Manizales',
    'types': [{'source': 'ror', 'type': 'Education'},
     {'source': 'openalex', 'type': 'education'}],
    'country': 'Colombia',
    'country_code': 'CO'},
   {'id': '059yx9a68',
    'name': 'Universidad Nacio

In [39]:
def get_oa_author_id(kid):
    pipe=[{
        "$match":{"_id":kid}
    },
        {
        "$project": {
            "external_ids": {
                "$filter": {
                    "input": "$external_ids",
                    "as": "ext",
                    "cond": {"$eq": ["$$ext.source", "openalex"]}
                }
            }
        }
    }
    ]
    _id = list(khp.aggregate(pipe))
    return _id[0]["external_ids"][0]["id"] if _id and _id[0]["external_ids"] else None


In [40]:
ws[0]["authors"]["id"]

'0000782025'

In [58]:
found = []
for w in ws[0:1000]:
    _id = get_oa_author_id(w["authors"]["id"])
    fw=oa.find_one({"id":w["external_ids"][0]["id"]},{"authorships":1})
    for a in fw["authorships"]:
        if a["author"]["id"] == _id and len(a["institutions"]) != len(w["authors"]["affiliations"]):
            print(len(a["institutions"]),len(w["authors"]["affiliations"]))
            #print(w["external_ids"][0]["id"])
            found.append(w)


1 2
1 2
1 2
1 2
1 2
1 2
1 2
1 2
1 2
3 2
3 2
3 2
1 2
1 3
1 2
1 2
3 2
1 2
6 4
1 2
1 2
1 3
1 4
1 3
3 2
1 4
1 3
1 2
1 2
0 2
1 4
3 2
1 2
1 2
1 2
1 2
1 2
3 2
3 2
3 2
1 2
1 2
1 2
1 2
1 2
2 4
1 2
1 2
1 2
1 2
1 2
1 2
3 2
3 2
4 2
4 2
3 2
4 2
4 2
1 2
4 2
3 2
3 2
3 2
3 2
3 2
4 2
3 2
3 2
3 2
3 2
5 3
3 2
3 2
3 2
3 2
4 2
4 2
3 2
3 2
1 2
1 2
1 2
3 2
4 3
1 2
1 2
1 2
1 2
1 2
1 2
1 2
1 2
1 2
3 2
3 2
3 2
3 2
3 2
3 2
3 2
4 2
4 2
3 2
4 2
4 2
3 2
3 2
3 2
3 2
3 2
4 2
3 2
3 2
3 2
3 2
3 2
3 2
4 2
1 2
1 2
1 2
1 2
1 2
1 2
1 2
5 3
5 3
1 3
4 2
3 2
3 2
4 2
4 2
3 2
4 2
4 2
1 2
4 2
3 2
3 2
3 2
3 2
3 2
3 2
4 2
3 2
3 2
3 2
3 2
5 3
3 2
3 2
3 2
3 2
4 2
4 2
4 2
3 2
3 2
1 2
3 2
3 2
3 2
3 2
3 2
3 2
3 2
3 2
3 2
3 2
3 2
1 2
1 3
1 3
3 2
1 2
1 3
1 2
1 2
1 2
3 2
3 2
4 2
4 2
3 2
4 2
4 2
1 2
4 2
3 2
3 2
3 2
3 2
4 2
3 2
3 2
3 2
4 2
4 2
3 2
3 2
3 2
4 2
1 2
1 2
1 2
1 2
4 2
4 2
4 2
4 2
4 2
4 2
4 2
4 2
4 2
4 2
1 2
1 2
3 2
3 2
4 2
4 2
3 2
4 2
4 2
3 2
3 2
3 2
3 2
3 2
3 2
4 2
3 2
3 2
3 2
3 2
5 3
3 2
3 2
3 2
3 2
4 2
4 2
4 2
5 3
3 2
3 2
1 2


In [59]:
len(found)

528

In [60]:
for w in found:
    w["_id"]=str(w["_id"])

In [61]:
f=open("comet.json","w")
f.write(json.dumps(found))
f.close()